In [ ]:
#xgboost

#ROC/AUC
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt

def paintROC(y_test1,y_pred1):
    fpr,tpr,threshold = roc_curve(y_test1, y_pred1) 
    roc_auc = auc(fpr,tpr) ###AUC

    plt.figure()
    lw = 2
    plt.figure(figsize=(7,7))
    plt.plot(fpr, tpr, color='navy',
            lw=lw, label='ROC curve (area = %0.2f)' % roc_auc) 
    plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('XGBoost_ROC_Curve')
    plt.legend(loc="lower right")
    plt.show()

import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
import sklearn
from sklearn import metrics
import time
from keras import layers
import xgboost as xgb
from sklearn import preprocessing
from xgboost.sklearn import XGBClassifier as xgbc

#Input data
#indata=pd.read_csv("../input/DGA_test.csv")
indata=pd.read_csv(r'E:\Data_test\DGA_test.csv')

X=indata['url']
labels=indata['label']

# Generate a dictionary of valid characters
valid_chars = {x:idx+1 for idx, x in enumerate(set(''.join(X)))}

max_features = len(valid_chars) + 1
maxlen = np.max([len(x) for x in X])
# Convert characters to int and pad
X = [[valid_chars[y] for y in x] for x in X]
X = sequence.pad_sequences(X, maxlen=maxlen)
y=np.array(labels)

#Standardazition
X=preprocessing.scale(X)

from sklearn.model_selection import StratifiedKFold  
cv_number=5# parameter
skf= StratifiedKFold(n_splits=cv_number) # x-fold cv 
skf.get_n_splits(X,y) 
print("CV method:StratifiedKFlod  Split number: ",cv_number) 

ac_test=[]
ac_train=[]
ac_test_auc=[]
ac_train_auc=[]
term=1

for train_index, test_index in skf.split(X, y):  
    print("Start:   current: ",term,"  ","Total: ",cv_number) 
    time_start=time.time()
    X_train,X_test = X[train_index], X[test_index]  
    y_train,y_test = y[train_index], y[test_index] 
    #train model on different training base
    time_start=time.time()
    
    model=xgbc(silent=0,
        learning_rate=0.3,
        min_child_weight=1,
        max_depth=9,
        gamma=0.3,
        subsample=0.9,
        max_delta_step=0,
        colsample_bytree=0.8,
        colsample_bylevel=1,
        reg_alpha=0.01,
        reg_lambda=1,
        scale_pos_weight=1,
        n_estimators=150,
        objective='binary:logistic',
        seed=0)
    model.fit(X_train,y_train)
    
    y_pred_test=model.predict_proba(X_test)[:,1]
    y_pred_train=model.predict_proba(X_train)[:,1]
    
#    ac_test.append(metrics.accuracy_score(y_test.astype('int'), y_pred_test.astype('int')))
 #   ac_train.append(metrics.accuracy_score(y_train.astype('int'),y_pred_train.astype('int')))
    ac_test_auc.append(sklearn.metrics.roc_auc_score(y_test, y_pred_test))
    ac_train_auc.append(sklearn.metrics.roc_auc_score(y_train,y_pred_train))
    
    time_end=time.time()
    print("Time:",round(time_end-time_start,4))
    term+=1
    
    paintROC(y_test,y_pred_test)
    
ac_test_auc=np.array(ac_test_auc)
ac_train_auc=np.array(ac_train_auc)
ac_test=np.array(ac_test)
ac_train=np.array(ac_train)
print("Mean Accuracy of Train Set: ",round(ac_train.mean(),4),
     " Mean Accuracy of the Test Set:",round(ac_test.mean(),4),
      " Mean AUC of the Train Set:",round(ac_train_auc.mean(),4),
      " Mean AUC of the Test Set:",round(ac_test_auc.mean(),4)
     )
    